In [1]:
import requests
import numpy as np
import pandas as pd
import pymysql as mysql
from pprint import pprint
from tabulate import tabulate
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from termcolor import colored
from sqlalchemy import create_engine, MetaData
from sqlalchemy_schemadisplay import create_schema_graph
from IPython.display import Image
import base64
warnings.filterwarnings('ignore')

APIs (rapid API), username trevsauer2023

In [55]:
city_id = "San Diego"

# Air Quality 

In [62]:
url = "https://air-quality-by-api-ninjas.p.rapidapi.com/v1/airquality"

querystring = {"city":city}

headers = {
	"X-RapidAPI-Key": "ad2cc6becemsh2e08af212b398c8p15d59djsn310073516954",
	"X-RapidAPI-Host": "air-quality-by-api-ninjas.p.rapidapi.com"
}

a_response = requests.request("GET", url, headers=headers, params=querystring)

# Convert the response to a dictionary
a_data = a_response.json()

# Create a Pandas dataframe from the dictionary
a_df = pd.DataFrame(a_data)
a_df

,CO,NO2,O3,SO2,PM2.5,PM10,overall_aqi
concentration,226.97,4.71,87.98,1.28,1.74,2.54,132
aqi,2.00,5.00,132.00,1.00,5.00,2.00,132


# Covid API

###  Coronavirus COVID-19 Statistics API Based on public data by Johns Hopkins CSSE. It includes search by country, province, and date. Returns diff for confirmed, deaths, and recovered, compared with the previous day.



In [56]:
# make the API call
# link to documentation: https://rapidapi.com/axisbits-axisbits-default/api/covid-19-statistics/
url = "https://covid-19-statistics.p.rapidapi.com/reports"

# optional parameter selected to querying the API
querystring = {"city_name":city_id}

# API credentials
headers = {
	"X-RapidAPI-Key": "ad2cc6becemsh2e08af212b398c8p15d59djsn310073516954",
	"X-RapidAPI-Host": "covid-19-statistics.p.rapidapi.com"
}

# get raw response query from the API
response = requests.request("GET", url, headers=headers, params=querystring)
print(response.text)

{"data":[{"date":"2023-02-23","confirmed":12080387,"deaths":100816,"recovered":0,"confirmed_diff":21022,"deaths_diff":231,"recovered_diff":0,"last_update":"2023-02-24 04:20:54","active":11979571,"active_diff":20791,"fatality_rate":0.0083,"region":{"iso":"USA","name":"US","province":"California","lat":"36.1162","long":"-119.6816","cities":[{"name":"San Diego","date":"2023-02-23","fips":6073,"lat":"33.03484597","long":"-116.7365326","confirmed":1063940,"deaths":5768,"confirmed_diff":1914,"deaths_diff":9,"last_update":"2023-02-24 04:20:54"}]}}]}


In [58]:
# convert the response to a dictionary
data = response.json()

# create a Pandas dataframe from the dictionary
df = pd.DataFrame(data)

# unpack the dictionaries in the column into separate columns
df_unpacked = pd.json_normalize(df['data'])

# merge the new columns back into the original dataframe
df_1 = df.merge(df_unpacked, left_index=True, right_index=True)

# drop the original column containing the dictionaries
df_2 = df_1.drop('data', axis=1)

# extract the dictionary from region.cities column for flattening
col_dic = df_2['region.cities'].iloc[0]

# turn the extracted dictionary into its on dataframe 
flatten_col = pd.DataFrame(col_dic)

# merge the two dataframes
result = df_2.merge(flatten_col, left_index=True, right_index=True)

# drop columns 
result.drop(['last_update_x', 'active', 'active_diff', 'region.name', 'region.cities', 'date_y'], axis=1, inplace=True)

# rename columns to be more understandable
final_output_df = result.rename(columns={"date_x": "date", "confirmed_x": "total_confirmed_cases_state","deaths_x": "total_deaths_state", 
                                         "confirmed_diff_x": "confirmed_today_state", "deaths_diff_x": "deaths_today_state",
                                        "recovered_diff": "recovered_today_state", "fatality_rate":"fatality_rate_state","region.iso":"country",
                                        "region.province": "state", "region.lat":"state_latitude", "region.long":"state_longitude", "name": "city",
                                        "fips":"fips_city_code", "lat": "city_latitude", "long": "city_longitude", "confirmed_y":"total_confirmed_cases_city",
                                        "deaths_y": "total_deaths_city", "confirmed_diff_y":"confirmed_today_city", "deaths_diff_y": "deaths_today_city",
                                        "last_update_y": "last_API_update"})

# show the values of all the columns
row_values = final_output_df.iloc[0].to_dict()
row_values

{'date': '2023-02-23',
 'total_confirmed_cases_state': 12080387,
 'total_deaths_state': 100816,
 'recovered': 0,
 'confirmed_today_state': 21022,
 'deaths_today_state': 231,
 'recovered_today_state': 0,
 'fatality_rate_state': 0.0083,
 'country': 'USA',
 'state': 'California',
 'state_latitude': '36.1162',
 'state_longitude': '-119.6816',
 'city': 'San Diego',
 'fips_city_code': 6073,
 'city_latitude': '33.03484597',
 'city_longitude': '-116.7365326',
 'total_confirmed_cases_city': 1063940,
 'total_deaths_city': 5768,
 'confirmed_today_city': 1914,
 'deaths_today_city': 9,
 'last_API_update': '2023-02-24 04:20:54'}

# Weather API(s)

In [60]:
url = "https://weather-by-api-ninjas.p.rapidapi.com/v1/weather"

headers = {
	"X-RapidAPI-Key": "570296f95emsh5f009ea13d4ed2cp106b38jsnbd81b0645b00",
	"X-RapidAPI-Host": "weather-by-api-ninjas.p.rapidapi.com"
}
querystring = {"city":city_id}

w_response = requests.request("GET", url, headers=headers, params = querystring)

# Convert the response to a dictionary
w_data = w_response.json()

# Create a Pandas dataframe from the dictionary
w_df = pd.DataFrame(w_data, index = [0])
w_df

,cloud_pct,temp,feels_like,humidity,min_temp,max_temp,wind_speed,wind_degrees,sunrise,sunset
0,100,15,14,64,13,16,6.71,144,1677248554,1677289290


In [13]:
url = "https://crime-data-by-zipcode-api.p.rapidapi.com/crime_data"

querystring = {"zip":"94109"}

headers = {
	"X-RapidAPI-Key": "ad2cc6becemsh2e08af212b398c8p15d59djsn310073516954",
	"X-RapidAPI-Host": "crime-data-by-zipcode-api.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

df = pd.DataFrame(response)

print(response.text)


{"Overall":{"Zipcode":"94109","Overall Crime Grade":"F","Violent Crime Grade":"F","Property Crime Grade":"F","Other Crime Grade":"F","Fact":"A crime occurs every 1 hour 11 minutes (on average) in 94109.","Risk":"300%","Risk Detail":"Your home is 300% more likely to be robbed with no home security system."},"Crime BreakDown":[{"Violent Crime Rates":{"Assault":"11.35","Robbery":"4.370","Rape":"0.4715","Murder":"0.0509"},"0":{"Total Violent Crime":"16.24","Total Violent Crime Score":"(F)"}},{"Property Crime Rates":{"Theft":"59.22","Vehicle Theft":"3.198","Burglary":"12.50","Arson":"3.770"},"0":{"Total Property Crime":"78.69","Total Property Crime Score":"(F)"}},{"Other Crime Rates":{"Kidnapping":"0.4780","Drug Crimes":"25.45","Vandalism":"7.619","Identity Theft":"1.047","Animal Cruelty":"0.0570"},"0":{"Total Other Rate":"34.65","Total Other Score":"(F)"}}],"Crime Rates Nearby":[{"Nearby Zip":"94108, CA","Overall Crime Grade":"F","Violent Crime Grade":"F","Property Crime Grade":"F"},{"Near